# 🧠 ECLASS Code Mapping with Fuzzy Matching (RapidFuzz & Levenshtein)

This notebook demonstrates how to match and migrate ECLASS classification codes between versions using fuzzy string matching techniques like Levenshtein distance and RapidFuzz.


In [1]:
from rapidfuzz import process, distance

In [2]:
import pandas as pd

In [3]:
from nltk.tokenize import word_tokenize

In [4]:
import os

In [5]:
import re

In [6]:
import numpy as np

## Upload and Standartize your data

In [7]:
# Insert file path to your products and/or eclasses of interest
# "R:\doc\Kunden Produktlisten\!_current working file\working_files_JAN 2025\EURproductCatalogExpanded.xls"

my_catalog_file = pd.read_excel(input('Paste file path: '))

In [8]:
eclass_n_1 = pd.read_excel(my_catalog_file)

In [9]:

def detect_eclass_columns(eclass_n_1):
    eclass_pattern = re.compile(r'^\d{2}(-\d{2}){1,3}$|^\d{8}$')
    candidate_cols = []

    for col in eclass_n_1.columns:
        col_lower = col.lower()
        name_match = any(keyword in col_lower for keyword in ["eclass", "ecl@ss"])

        sample = eclass_n_1[col].dropna().astype(str)
        if len(sample) >= 377:
            sample = sample.sample(n=377, random_state=42)

        match_ratio = sample.apply(lambda x: bool(eclass_pattern.match(x))).mean()
        uniqueness_ratio = sample.nunique() / len(sample) if len(sample) > 0 else 0

        probability_score = match_ratio + 0.2 * uniqueness_ratio  # Weight uniqueness modestly

        if probability_score > 0.75 and name_match:
            candidate_cols.append(col)
            print(f"{col}: match = {match_ratio:.2f}, uniqueness = {uniqueness_ratio:.2f}, score = {probability_score:.2f}")

    return candidate_cols


In [10]:
column_name = detect_eclass_columns(eclass_n_1)

print("Detected ECLASS columns:", column_name)

ECLASS: match = 1.00, uniqueness = 0.03, score = 1.01
Detected ECLASS columns: ['ECLASS']


In [11]:
mycat_e = set(eclass_n_1['{}'.format(str(column_name).replace('"','').replace("'",'').replace('[','').replace(']',''))])

In [12]:
eclass_n_1.head()

,QTYAMOUNT,VOLUMEPRICE,PARTNUM,PTITLE,CURR,STOCK,PRODUCTIMAGE,PAGELINK,UNITS,LEVEL1,...,LEVEL5,LEVEL6,LEVEL7,PRODUCTWEIGHT,BUNIT,UNSPSCCODE,ECLASS,KEYWORDS,SHORTTITLE,COO
0,1,11.32,10125A,"Standard SMA905 Multimode Connector, Ø128 µm B...",Euro,318.0,http://www.thorlabs.com/images/small/1046-sm.jpg,http://www.thorlabs.com/newgrouppage9.cfm?obje...,Each,Thorlabs Products,...,Multimode Fiber Optic Connectors,NaN,NaN,0.014991,BU_OPTICS,43223310,32020000,"125 um,125um,128 um,128um,adapt,adapter,adapto...",SMA 905 Connector: MM 125um,JPN
1,1,12.98,10125H,"Hex Nut SMA905 Multimode Connector, Ø128 µm Bo...",Euro,24.0,http://www.thorlabs.com/images/small/TTN201753...,http://www.thorlabs.com/newgrouppage9.cfm?obje...,Each,Thorlabs Products,...,Multimode Fiber Optic Connectors,NaN,NaN,0.035053,BU_OPTICS,43223310,32020000,"125 um,125um,128 um,128um,adapt,adapter,adapto...",NaN,JPN
2,1,418.98,10125HG,Fiber Optic SMA905 Connector Height Gauge,Euro,3.0,http://www.thorlabs.com/images/small/14004-sm.jpg,http://www.thorlabs.com/newgrouppage9.cfm?obje...,EACH,Thorlabs Products,...,NaN,NaN,NaN,0.345988,BU_OPTICS,41111614,27061003,"test,depth,check,standard,measure,polishing,po...",Fiber Optic SMA Connector Height Gauge,USA
3,1,8.75,10127C,"ST/PC Multimode Connector, &#216;127 &#956;m B...",Euro,40.0,http://www.thorlabs.com/images/small/TTN018640...,http://www.thorlabs.com/newgrouppage9.cfm?obje...,EACH,Thorlabs Products,...,Multimode Fiber Optic Connectors,NaN,NaN,0.013999,BU_OPTICS,43223310,27069290,"bayonet,connectorization,ferrule,ceramic,127 b...","ST/PC Connector: MM 127um +4/0um, Ceramic Ferrule",MEX
4,1,22.23,10128V,"High-Temp., Vacuum-Compatible SMA905 MM Connec...",Euro,6.0,http://www.thorlabs.com/images/small/TTN159010...,http://www.thorlabs.com/newgrouppage9.cfm?obje...,Each,Thorlabs Products,...,Multimode Fiber Optic Connectors,NaN,NaN,0.350531,BU_OPTICS,43223310,32020000,"304 SS,304 stainless steel,connector,fiber con...",NaN,HKG


In [13]:
# which versions are you going to match? e.g. Old: 5.1 and New: 5.1.4

versionX = str(input('Insert old version: '))
versionY = str(input('Insert new version: '))

Insert old version: 5.1
Insert new version: 6.2


# File with Old ECLASS Version (X)

In [14]:
# Insert your file path, sheet name and rows to skip
# 'R://doc//Kunden Produktlisten//XX_ECLASS//5-10 Eclass categories.xlsx'
# 'Tabelle2'
# 3

eclass_n1_kw = pd.read_excel(input('Paste file path to ECLASS version X: '),
                             sheet_name=input('Paste sheet name: '), skiprows=3)

In [15]:
eclass_n1_kw.columns

Index(['eclass5.1 categorie (PreferredName)', 'ECLASS5.1',
       'eclass6.0 categorie (PreferredName)', 'ECLASS6.0', 'name difference'],
      dtype='object')

In [16]:
eclass_n1_kw.head()

,eclass5.1 categorie (PreferredName),ECLASS5.1,eclass6.0 categorie (PreferredName),ECLASS6.0,name difference
0,"Machine, device (for special applications)",17000000,NaN,NaN,NaN
1,Food processing machinery,17010000,NaN,NaN,NaN
2,Bakery machinery,17010100,NaN,NaN,NaN
3,Bakery machinery (unclassified),17010190,NaN,NaN,NaN
4,Meat processing,17010200,NaN,NaN,NaN


In [19]:
def find_eclass_and_preferred_columns(df, versionX):
    eclass_col = None
    preferred_col = None

    # Normalize versionX to lowercase string for matching
    versionX = str(versionX)

    for col in df.columns:
        col_lower = col.lower()

        # Check for ECLASS column: name contains 'eclass' or 'ecl@ss' AND versionX, and values are mostly 8-digit numbers
        if any(kw in col_lower for kw in ["eclass", "ecl@ss"]) and versionX in col_lower:
            sample1 = df[col].dropna().astype(str).head(200)
            if sample1.apply(lambda x: bool(re.fullmatch(r'\d{8}', x))).mean() > 0.6:
                eclass_col = col

        # Check for Preferred Value column
        if df[col].dropna().astype(str).map(len).gt(9).mean() > 0.9:
            preferred_col = col

    return eclass_col, preferred_col

In [20]:
eclass_col, preferred_col = find_eclass_and_preferred_columns(eclass_n1_kw, versionX)

print("ECLASS column:", eclass_col)
print("PreferredName column:", preferred_col)

eclass_n1_kw = eclass_n1_kw.rename(columns={f'{preferred_col}':f'NAME{versionX}'})
eclass_n1_kw = eclass_n1_kw.rename(columns={f'{eclass_col}':f'ECLASS{versionX}'})

eclass_n1_kw.head()

ECLASS column: ECLASS5.1
PreferredName column: eclass5.1 categorie (PreferredName)


,NAME5.1,ECLASS5.1,eclass6.0 categorie (PreferredName),ECLASS6.0,name difference
0,"Machine, device (for special applications)",17000000,NaN,NaN,NaN
1,Food processing machinery,17010000,NaN,NaN,NaN
2,Bakery machinery,17010100,NaN,NaN,NaN
3,Bakery machinery (unclassified),17010190,NaN,NaN,NaN
4,Meat processing,17010200,NaN,NaN,NaN


In [21]:
eclass_n1_kw = eclass_n1_kw[eclass_n1_kw['ECLASS5.1'].isin(set(eclass_n_1.ECLASS))]

In [22]:
eclass_n1_kw.shape

(22, 5)

In [23]:
# delete all columns ecxept ECLASS number and NAME

for col in eclass_n1_kw.columns:
    if col.startswith('ECLASS{}'.format(versionX)):
        pass
    elif col.startswith('NAME{}'.format(versionX)):
        pass
    else:
        eclass_n1_kw = eclass_n1_kw.drop(columns=col)
        print(col, ' deleted')

eclass6.0 categorie (PreferredName)  deleted
ECLASS6.0  deleted
name difference  deleted


In [24]:
eclass_n1_kw[f'Tokens{versionX}'] = eclass_n1_kw[f'NAME{versionX}'].apply(lambda x: re.findall(r'\w+', x))

In [25]:
eclass_n1_kw

,NAME5.1,ECLASS5.1,Tokens5.1
2693,"Measuring tool (surveying, unclassified)",21160190,"[Measuring, tool, surveying, unclassified]"
2796,Measuring machine (unclassified),21170590,"[Measuring, machine, unclassified]"
3979,"Screw (special, unclassified)",23110690,"[Screw, special, unclassified]"
4570,PVC adhesive tape,23330201,"[PVC, adhesive, tape]"
8046,Power supply,27040000,"[Power, supply]"
8116,Fiber-optic duplex patch cable,27061003,"[Fiber, optic, duplex, patch, cable]"
8121,Coaxial cable,27061802,"[Coaxial, cable]"
8122,Fibre optic cable,27061803,"[Fibre, optic, cable]"
8159,"Cable, line (accessories, unclassified)",27069290,"[Cable, line, accessories, unclassified]"
8278,LED-module,27110635,"[LED, module]"


## File with New ECLASS Version 

In [26]:
fileY = r"{}".format(input('Paste file path for target version of ECLASS csv: '))

In [27]:
eclass_n2_kw = pd.read_csv(fileY, 
    delimiter=';'
)


C:\Users\hahafonova\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
eclass_n2_kw.columns

Index(['supplier', 'idcl', 'identifier', 'version number', 'publication date',
       'revision number', 'coded name', 'preferred name', 'definition',
       'iso language code', 'iso country code', 'note', 'remark', 'level',
       'mksubklass', 'mksynonym', 'mkbsa', 'IRDI_CC', 'type of SE',
       'correction'],
      dtype='object')

In [29]:

def find_eclass_and_keywords(df, versionY):
    class_col = None
    keyword_col = None

    # Normalize versionY to lowercase string for matching
    versionY = str(versionY)

    for col in df.columns:
        col_lower = col.lower()

        # Check for ECLASS column: name contains 'eclass' or 'ecl@ss' AND versionX, and values are mostly 8-digit numbers
        if any(kw in col_lower for kw in ["class", "coded"]) or versionY in col_lower:
            sample = df[col].dropna().astype(str).head(20)
            if sample.apply(lambda x: bool(re.fullmatch(r'\d{8}', x))).mean() > 0.6:
                class_col = col

        # Check for Preferred Value column
        if df[col].dropna().astype(str).map(len).gt(10).mean() > 0.9 and all(kw in col_lower for kw in ["preferred name"]):
            keyword_col = col

    return class_col, keyword_col


In [30]:
class_col, keyword_col = find_eclass_and_keywords(eclass_n2_kw, versionY)

print("ECLASS column:", class_col)
print("Key Word column:", keyword_col)

eclass_n2_kw = eclass_n2_kw.rename(columns={f'{keyword_col}':f'NAME{versionY}'})
eclass_n2_kw = eclass_n2_kw.rename(columns={f'{class_col}':f'ECLASS{versionY}'})

eclass_n2_kw.head()

ECLASS column: coded name
Key Word column: preferred name


,supplier,idcl,identifier,version number,publication date,revision number,ECLASS6.2,NAME6.2,definition,iso language code,iso country code,note,remark,level,mksubklass,mksynonym,mkbsa,IRDI_CC,type of SE,correction
0,0173-1,ACE546002,ACE546,2,2008-12-03,1,16000000,"Food, beverage, tobacco",NaN,en,EN,NaN,NaN,1,1,NaN,NaN,0173-1#01-ACE546#002,1,False
1,0173-1,ACE547002,ACE547,2,2008-12-03,1,16010000,"Cereal, pulse",NaN,en,EN,NaN,NaN,2,1,NaN,NaN,0173-1#01-ACE547#002,1,False
2,0173-1,ACE561002,ACE561,2,2008-12-03,1,16010100,Cereal,NaN,en,EN,NaN,NaN,3,1,NaN,NaN,0173-1#01-ACE561#002,1,False
3,0173-1,ACE794004,ACE794,4,2009-11-28,1,16010101,Barley,NaN,en,EN,NaN,NaN,4,0,NaN,NaN,0173-1#01-ACE794#004,1,False
4,0173-1,ACE796004,ACE796,4,2009-11-28,1,16010102,Oat,NaN,en,EN,NaN,NaN,4,0,NaN,NaN,0173-1#01-ACE796#004,1,False


In [31]:
# delete all columns ecxept ECLASS number and NAME

for col in eclass_n2_kw.columns:
    if col.startswith('ECLASS{}'.format(versionY)):
        pass
    elif col.startswith('NAME{}'.format(versionY)):
        pass
    else:
        eclass_n2_kw = eclass_n2_kw.drop(columns=col)
        print(col, ' deleted')

supplier  deleted
idcl  deleted
identifier  deleted
version number  deleted
publication date  deleted
revision number  deleted
definition  deleted
iso language code  deleted
iso country code  deleted
note  deleted
remark  deleted
level  deleted
mksubklass  deleted
mksynonym  deleted
mkbsa  deleted
IRDI_CC  deleted
type of SE  deleted
correction  deleted


In [32]:
eclass_n2_kw

,ECLASS6.2,NAME6.2
0,16000000,"Food, beverage, tobacco"
1,16010000,"Cereal, pulse"
2,16010100,Cereal
3,16010101,Barley
4,16010102,Oat
...,...,...
32827,41149090,"Services (marketing, other, unclassified)"
32828,90000000,Interim class (unclassified)
32829,90900000,Interim classification (unspecified)
32830,90909000,Interim classification (unspecified)


In [33]:
# delete all columns ecxept ECLASS number and NAME

for col in eclass_n2_kw.columns:
    if col.startswith('ECLASS{}'.format(versionY)):
        pass
    elif col.startswith('NAME{}'.format(versionY)):
        pass
    else:
        eclass_n2_kw = eclass_n2_kw.drop(columns=col)
        print(col, ' deleted')

In [34]:
eclass_n2_kw

,ECLASS6.2,NAME6.2
0,16000000,"Food, beverage, tobacco"
1,16010000,"Cereal, pulse"
2,16010100,Cereal
3,16010101,Barley
4,16010102,Oat
...,...,...
32827,41149090,"Services (marketing, other, unclassified)"
32828,90000000,Interim class (unclassified)
32829,90900000,Interim classification (unspecified)
32830,90909000,Interim classification (unspecified)


In [35]:
eclass_n2_kw[f'Tokens{versionY}'] = eclass_n2_kw[f'NAME{versionY}'].apply(lambda x: re.findall(r'\w+', x))

In [36]:
eclass_n2_kw

,ECLASS6.2,NAME6.2,Tokens6.2
0,16000000,"Food, beverage, tobacco","[Food, beverage, tobacco]"
1,16010000,"Cereal, pulse","[Cereal, pulse]"
2,16010100,Cereal,[Cereal]
3,16010101,Barley,[Barley]
4,16010102,Oat,[Oat]
...,...,...,...
32827,41149090,"Services (marketing, other, unclassified)","[Services, marketing, other, unclassified]"
32828,90000000,Interim class (unclassified),"[Interim, class, unclassified]"
32829,90900000,Interim classification (unspecified),"[Interim, classification, unspecified]"
32830,90909000,Interim classification (unspecified),"[Interim, classification, unspecified]"


## RAPIDFUZZ

In [ ]:
n = input('Select number of candidates pro each ECLASS')

In [37]:


# Function to find top 3 matches along with their ECLASS514 values
def find_top_matches_with_eclass(token51, table2):
    matches = process.extract(token51, table2[f'Tokens{versionY}'], limit=n)  # Get top N matches
    # Append the corresponding eclass value to the matches
    detailed_matches = [
        (match[0], match[1], match[2], table2.iloc[match[2]][f'ECLASS{versionY}'])  
        for match in matches
    ]
    return detailed_matches

eclass_n1_kw['TopNMatches'] = eclass_n1_kw[f'Tokens{versionX}'].apply(lambda x: find_top_matches_with_eclass(x, eclass_n2_kw))

# Print the results
for index, row in eclass_n1_kw.iterrows():
    print(f"ECLASS {versionX}: {row[f'ECLASS{versionX}']} - {row[f'Tokens{versionX}']} - Top N Matches:")
    for match in row['TopNMatches']:
        print(f"  Match: {match[0]}, Score: {match[1]:.2f}, ECLASS{versionY}: {match[3]}")


ECLASS 5.1: 21160190 - ['Measuring', 'tool', 'surveying', 'unclassified'] - Top 3 Matches:
  Match: ['Measuring', 'tool', 'surveying', 'unclassified'], Score: 100.00, ECLASS6.2: 21160190
  Match: ['Measuring', 'tool'], Score: 90.00, ECLASS6.2: 21170600
  Match: ['Measuring', 'tool', 'unclassified'], Score: 85.71, ECLASS6.2: 21170690
ECLASS 5.1: 21170590 - ['Measuring', 'machine', 'unclassified'] - Top 3 Matches:
  Match: ['Measuring', 'machine', 'unclassified'], Score: 100.00, ECLASS6.2: 21170590
  Match: ['Measuring', 'machine'], Score: 90.00, ECLASS6.2: 21170500
  Match: ['Doubling', 'twisting', 'winding', 'reeling', 'machine', 'unclassified'], Score: 72.00, ECLASS6.2: 17020890
ECLASS 5.1: 23110690 - ['Screw', 'special', 'unclassified'] - Top 3 Matches:
  Match: ['Screw', 'special', 'unclassified'], Score: 100.00, ECLASS6.2: 23110690
  Match: ['Screw', 'special'], Score: 90.00, ECLASS6.2: 23110600
  Match: ['Nut', 'special', 'unclassified'], Score: 66.67, ECLASS6.2: 23110890
ECLASS 5

In [38]:
for e in eclass_n1_kw['TopNMatches']:
    print(e[0][-1])

21160190
21170590
23110690
23330201
27040000
27061003
27061802
27061803
27069290
27110635
27110636
27200307
27201304
27230218
27230227
27270905
27272704
32020100
32020100
32020103
20399090
36610408


In [39]:
eclass_n1_kw.shape

(22, 4)

In [40]:

df = eclass_n1_kw.explode('TopNMatches')

In [41]:
df

,NAME5.1,ECLASS5.1,Tokens5.1,Top3Matches
2693,"Measuring tool (surveying, unclassified)",21160190,"[Measuring, tool, surveying, unclassified]","([Measuring, tool, surveying, unclassified], 1..."
2693,"Measuring tool (surveying, unclassified)",21160190,"[Measuring, tool, surveying, unclassified]","([Measuring, tool], 90.0, 4423, 21170600)"
2693,"Measuring tool (surveying, unclassified)",21160190,"[Measuring, tool, surveying, unclassified]","([Measuring, tool, unclassified], 85.714285714..."
2796,Measuring machine (unclassified),21170590,"[Measuring, machine, unclassified]","([Measuring, machine, unclassified], 100.0, 44..."
2796,Measuring machine (unclassified),21170590,"[Measuring, machine, unclassified]","([Measuring, machine], 90.0, 4403, 21170500)"
...,...,...,...,...
13926,"Consumable material (other, laboratory, unclas...",32039090,"[Consumable, material, other, laboratory, uncl...","([Pipe, and, pipe, fitting, building, material..."
13926,"Consumable material (other, laboratory, unclas...",32039090,"[Consumable, material, other, laboratory, uncl...","([Equipment, for, odd, lot, storage, hazardous..."
17402,Laser beam source (machine tool),36610408,"[Laser, beam, source, machine, tool]","([Laser, beam, source, machine, tool], 100.0, ..."
17402,Laser beam source (machine tool),36610408,"[Laser, beam, source, machine, tool]","([Thermal, beam, processing, system, a, beam, ..."


In [42]:
df['Match_keyword'] = df['TopNMatches'].apply(lambda e: e[0]).apply(lambda x : ' '.join(x))

In [43]:
df

,NAME5.1,ECLASS5.1,Tokens5.1,Top3Matches,Match_keyword
2693,"Measuring tool (surveying, unclassified)",21160190,"[Measuring, tool, surveying, unclassified]","([Measuring, tool, surveying, unclassified], 1...",Measuring tool surveying unclassified
2693,"Measuring tool (surveying, unclassified)",21160190,"[Measuring, tool, surveying, unclassified]","([Measuring, tool], 90.0, 4423, 21170600)",Measuring tool
2693,"Measuring tool (surveying, unclassified)",21160190,"[Measuring, tool, surveying, unclassified]","([Measuring, tool, unclassified], 85.714285714...",Measuring tool unclassified
2796,Measuring machine (unclassified),21170590,"[Measuring, machine, unclassified]","([Measuring, machine, unclassified], 100.0, 44...",Measuring machine unclassified
2796,Measuring machine (unclassified),21170590,"[Measuring, machine, unclassified]","([Measuring, machine], 90.0, 4403, 21170500)",Measuring machine
...,...,...,...,...,...
13926,"Consumable material (other, laboratory, unclas...",32039090,"[Consumable, material, other, laboratory, uncl...","([Pipe, and, pipe, fitting, building, material...",Pipe and pipe fitting building material other ...
13926,"Consumable material (other, laboratory, unclas...",32039090,"[Consumable, material, other, laboratory, uncl...","([Equipment, for, odd, lot, storage, hazardous...",Equipment for odd lot storage hazardous materi...
17402,Laser beam source (machine tool),36610408,"[Laser, beam, source, machine, tool]","([Laser, beam, source, machine, tool], 100.0, ...",Laser beam source machine tool
17402,Laser beam source (machine tool),36610408,"[Laser, beam, source, machine, tool]","([Thermal, beam, processing, system, a, beam, ...",Thermal beam processing system a beam source m...


In [44]:
df['Match_Score'] = df['TopNMatches'].apply(lambda e : e[1]).astype(int)

In [45]:

df['ECLASS_n'] = df['TopNMatches'].apply(lambda e : e[-1])

In [46]:
df.head(20)

,NAME5.1,ECLASS5.1,Tokens5.1,Top3Matches,Match_keyword,Match_Score,ECLASS_n
2693,"Measuring tool (surveying, unclassified)",21160190,"[Measuring, tool, surveying, unclassified]","([Measuring, tool, surveying, unclassified], 1...",Measuring tool surveying unclassified,100,21160190
2693,"Measuring tool (surveying, unclassified)",21160190,"[Measuring, tool, surveying, unclassified]","([Measuring, tool], 90.0, 4423, 21170600)",Measuring tool,90,21170600
2693,"Measuring tool (surveying, unclassified)",21160190,"[Measuring, tool, surveying, unclassified]","([Measuring, tool, unclassified], 85.714285714...",Measuring tool unclassified,85,21170690
2796,Measuring machine (unclassified),21170590,"[Measuring, machine, unclassified]","([Measuring, machine, unclassified], 100.0, 44...",Measuring machine unclassified,100,21170590
2796,Measuring machine (unclassified),21170590,"[Measuring, machine, unclassified]","([Measuring, machine], 90.0, 4403, 21170500)",Measuring machine,90,21170500
2796,Measuring machine (unclassified),21170590,"[Measuring, machine, unclassified]","([Doubling, twisting, winding, reeling, machin...",Doubling twisting winding reeling machine uncl...,72,17020890
3979,"Screw (special, unclassified)",23110690,"[Screw, special, unclassified]","([Screw, special, unclassified], 100.0, 6784, ...",Screw special unclassified,100,23110690
3979,"Screw (special, unclassified)",23110690,"[Screw, special, unclassified]","([Screw, special], 90.0, 6780, 23110600)",Screw special,90,23110600
3979,"Screw (special, unclassified)",23110690,"[Screw, special, unclassified]","([Nut, special, unclassified], 66.666666666666...",Nut special unclassified,66,23110890
4570,PVC adhesive tape,23330201,"[PVC, adhesive, tape]","([PVC, adhesive, tape], 100.0, 7482, 23330201)",PVC adhesive tape,100,23330201


In [47]:
df.to_excel('{}'.format(input('Paste file path to save extended results')), index=False)

## Last Version 

In [48]:
eclass_n1_kw['ECLASS_X'] = eclass_n1_kw['TopNMatches'].apply(
    lambda e : e[0][-1] )

In [49]:
eclass_n1_kw['Similarity'] = eclass_n1_kw['TopNMatches'].apply(
    lambda e : e[0][1] )

In [50]:
eclass_n1_kw = pd.merge(eclass_n1_kw, eclass_n2_kw, how='left', on='ECLASS_X')

In [51]:
eclass_n1_kw.Similarity.max()

100.0

In [52]:
eclass_n1_kw.describe()

,ECLASS5.1,ECLASS6.2,Similarity
count,2.200000e+01,2.200000e+01,22.000000
mean,2.756513e+07,2.703465e+07,95.189394
std,3.673591e+06,3.833226e+06,12.456092
min,2.116019e+07,2.039909e+07,60.000000
25%,2.706120e+07,2.704525e+07,100.000000
50%,2.715547e+07,2.711064e+07,100.000000
75%,2.727225e+07,2.726074e+07,100.000000
max,3.661041e+07,3.661041e+07,100.000000


In [28]:
eclass_n1_kw.to_excel('{}'.format(input('Paste path to save matched version')), index=False)